# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'capstone-prj-hyper-parameter'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'capstone-prj-cc'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2,slack_factor=0.2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'C': choice(0.01, 0.1, 1, 10, 100),
                                        'max_iter' : choice(50,75,100,125,150,175,200),
                                        'solver' : choice('liblinear','sag','lbfgs', 'saga')})

if "training" not in os.listdir():
    os.mkdir("./training")
shutil.copy('./train.py','./training')

sklearn_env = Environment.from_conda_specification(name="sklearn-env",file_path="./conda_dependencies.yaml")
#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='./training', compute_target = compute_target,script= 'train.py',environment= sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator, hyperparameter_sampling=param_sampling,policy=early_termination_policy,max_total_runs=25,
                                    max_duration_minutes=30,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [ ]:
#TODO: Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperDrive_run).show()
hyperDrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run

In [ ]:
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])

In [ ]:
# Code below registers the best model with the information of Metrics
model = best_run.register_model(model_name='HyperDrive_HighAccuracy', model_path='outputs/', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

inference_config = InferenceConfig(entry_script="score.py", environment = sklearn_env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1,enable_app_insights = True)

In [ ]:
service=Model.deploy(workspace=ws,
                    name="heart-hyper-deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
tosend = test[:2].values.tolist()
tosend = [tosend[0][:-1],tosend[1][:-1]]

In [ ]:
import requests
import json

data = json.dumps({"data":tosend})
headers = {'Content-Type':'application/json'}

response = requests.post(scoring_uri,data,headers=headers)
print(response.text)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
print('Status code:',response.status_code)

print(service.get_logs())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()
compute_target.delete()

In [ ]:
model.delete()